<a href="https://colab.research.google.com/github/SourLemon23/covid19-diagnosis/blob/master/COVID_19_CT_Diagnosis_V5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# # Grad-CAM
# ! pip install keras==2.2.0
# ! pip install tensorflow==1.10.0
# ! pip install keras==2.2.2 # EfficientNet PyPi
# ! pip install tensorflow==1.12.0 # EfficientNet PyPi

# Versions that are compatible with vis
# ! pip install keras==2.2.4
# ! pip install tensorflow==1.14.0
# ! pip install keras_applications >= 1.0.7
# https://stackoverflow.com/questions/57773636/no-attribute-set-keras-submodules

# ! pip install vis
# ! pip install scipy==1.1.0

In [3]:
import os
import numpy as np
import cv2
import datetime
import random

import tensorflow as tf
try:
    %tensorflow_version 2.x
except:
    pass

print('Using TensorFlow Version:', tf.__version__)

from tensorflow.keras import Input
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.layers import Conv2D, GlobalAveragePooling2D, GlobalMaxPooling2D, MaxPooling2D, BatchNormalization, Dense, Flatten, Dropout
from tensorflow.keras.metrics import Precision, Recall
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, LearningRateScheduler, TensorBoard

# # Clear any logs from previous runs
# rm -rf ./logs/
# Load the TensorBoard notebook extension
%load_ext tensorboard

import matplotlib.pyplot as plt

# EfficientNetB7
! pip install -U efficientnet
from efficientnet.tfkeras import EfficientNetB7
# from keras.applications.inception_v3 import preprocess_input

import scipy
print('Using SciPy Version:', scipy.__version__) # Should be 1.1.0

Using TensorFlow Version: 2.3.0
     |████████████████████████████████| 51kB 8.1MB/s 
Using SciPy Version: 1.4.1


In [4]:
# Make sure hardware accelerator is set to "GPU"
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
    print('GPU device not found')
else:
    print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [5]:
# Authorize access to mount Google Drive 
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
# Initialize dataset directories (Google Drive)
ct_training_dir   = r'/content/drive/My Drive/Colab Notebooks/COVID-19 Diagnosis/covid19_ct_dataset V4/Training'
ct_validation_dir = r'/content/drive/My Drive/Colab Notebooks/COVID-19 Diagnosis/covid19_ct_dataset V4/Validation'
ct_testing_dir    = r'/content/drive/My Drive/Colab Notebooks/COVID-19 Diagnosis/covid19_ct_dataset V4/Testing'

# Saved Weights
checkpoint_filepath = 'covid19_ct_model.h5'

In [7]:
# Initialize constants

# Make constants all CAPS
classes = ['COVID-19 Positive', 'COVID-19 Negative']
# img_width, img_height = 150, 150
img_width, img_height = 224, 224
target_size = (img_width, img_height)
input_shape = (img_width, img_height, 3)

testing_set_start_index = -50
testing_set_end_index = 49

batch_size = 30
epochs = 10
dropout_rate = 0.2
metrics = ['accuracy',
           Precision(name='precision'),
           Recall(name='recall')]

COVID_THRESHOLD = 0.3
EQUIDDST_MEAS = 0.5
NORMAL_THRESHOLD = 0.7

PERCENTAGE_FACTOR = 100;

In [8]:
# From https://github.com/haydengunraj/COVIDNet-CT/blob/8599c2a87856326a18bbdf6ffa5987f9c1e64b4b/data_utils.py#L40
def body_contour(binary_image):
    """Helper function to get body contour"""
    contours = find_contours(binary_image)
    areas = [cv2.contourArea(cnt) for cnt in contours]
    body_idx = np.argmax(areas)
    return contours[body_idx]


def auto_body_crop(image, scale=1.0):
    """Roughly crop an image to the body region"""
    # Create initial binary image
    filt_image = cv2.GaussianBlur(image, (5, 5), 0)
    thresh = cv2.threshold(filt_image[filt_image > 0], 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[0]
    bin_image = np.uint8(filt_image > thresh)
    erode_kernel = np.ones((7, 7), dtype=np.uint8)
    bin_image = cv2.erode(bin_image, erode_kernel)

    # Find body contour
    body_cont = body_contour(bin_image).squeeze()

    # Get bbox
    xmin = body_cont[:, 0].min()
    xmax = body_cont[:, 0].max() + 1
    ymin = body_cont[:, 1].min()
    ymax = body_cont[:, 1].max() + 1

    # Scale to final bbox
    if scale > 0 and scale != 1.0:
        center = ((xmax + xmin)/2, (ymin + ymax)/2)
        width = scale*(xmax - xmin + 1)
        height = scale*(ymax - ymin + 1)
        xmin = int(center[0] - width/2)
        xmax = int(center[0] + width/2)
        ymin = int(center[1] - height/2)
        ymax = int(center[1] + height/2)

    return image[ymin:ymax, xmin:xmax], (xmin, ymin, xmax, ymax)

In [9]:
# Augment data
# MAKE COMMENTS ON RIGHT OF EACH AUGMENTATION!!!!!!!!!!!!!
training_data_gen = ImageDataGenerator(rescale=1./255,
                                       featurewise_center=False, # Set input mean to 0 over dataset
                                       samplewise_center=False, # Set each sample mean to 0
                                       featurewise_std_normalization=False, # Divide inputs by std of dataset
                                       samplewise_std_normalization=False, # Divide each input by its std
                                       horizontal_flip=True,
                                       vertical_flip=True,
                                       zoom_range=0.15,
                                       shear_range=0.15,
                                       rotation_range=360,
                                       width_shift_range=0.15,
                                       height_shift_range=0.15,
                                       validation_split=0.15)

validation_data_gen = ImageDataGenerator(rescale=1./255)

testing_data_gen = ImageDataGenerator(rescale=1./255)

In [ ]:
# Split data
training_generator = training_data_gen.flow_from_directory(ct_training_dir,
                                                           target_size=target_size,
                                                           class_mode='binary',
                                                           batch_size=batch_size,
                                                           shuffle=True)

#  SHUFFLE = TRUE??
validation_generator = validation_data_gen.flow_from_directory(ct_validation_dir,
                                                               target_size=target_size,
                                                               class_mode='binary',
                                                               batch_size=batch_size,
                                                               shuffle=False)

testing_generator = testing_data_gen.flow_from_directory(ct_testing_dir,
                                                         target_size = target_size,
                                                         class_mode='binary',
                                                         batch_size=batch_size,
                                                         shuffle=True)

In [ ]:
# base_model = EfficientNetB7(weights='imagenet',
#                             include_top=True)

# base_model.summary()

In [ ]:
# Create a neural network
def create_model():
    # Instantiate a base model with pre-trained weights
    base_model = EfficientNetB7(weights='imagenet',
                                include_top=False,
                                input_shape=input_shape)
    x = base_model.output

    # Add new classifier layers to end of base_model
    x = BatchNormalization()(x)
    x = GlobalAveragePooling2D()(x)
    # x = Dropout(0.3)(x)
    x = Flatten()(x)
    x = Dense(2560, activation="relu")(x)
    x = Dense(2560, activation="relu")(x)

    # A Dense classifier with a single unit (binary classification)
    predictions = Dense(1, activation="sigmoid")(x)

    # Freeze the base model
    for layer in base_model.layers:
        layer.trainable = False

    model = Model(inputs=base_model.inputs, outputs=predictions)

    return model

In [ ]:
# Print a summary of the network architecture
def print_model_summary(model):
    model.summary()

In [ ]:
# Configure model
def compile_model(model):
    model.compile(loss='binary_crossentropy',
                  optimizer='adam', # tf.keras.optimizers.RMSprop(lr=1e-4)
                  metrics=metrics)

In [ ]:
# def exponential_decay(lr0, s):
#         def exponential_decay_fn(epoch):
#             return lr0 * 0.1 **(epoch / s)
#     return exponential_decay_fn

In [ ]:
# Train the model
def fit_model(model):
    # Early Stopping Implementation from: https://github.com/jeffheaton/t81_558_deep_learning/blob/master/t81_558_class_03_4_early_stop.ipynb
    # Early Stopping Documentation: https://www.tensorflow.org/api_docs/python/tf/keras/callbacks/EarlyStopping
    # es_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
    #                                           min_delta=1e-3,
    #                                           patience=3,
    #                                           verbose=1,
    #                                           mode='auto')

    checkpoint_cb = ModelCheckpoint(checkpoint_filepath,
                                    save_best_only=True)

    early_stopping_cb = EarlyStopping(patience=3,
                                      restore_best_weights=True)

    # exponential_decay_fn = exponential_decay(0.01, 20)

    # lr_scheduler = LearningRateScheduler(exponential_decay_fn)

    # https://www.tensorflow.org/tensorboard/get_started
    log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
    tensorboard_cb = TensorBoard(log_dir=log_dir,
                                 histogram_freq=1)

    history = model.fit(training_generator,
                        epochs=epochs,
                        steps_per_epoch=(training_generator.n/batch_size),
                        validation_data=validation_generator,
                        validation_steps=(validation_generator.n/batch_size),
                        callbacks=[checkpoint_cb, early_stopping_cb, tensorboard_cb])
    
    model.load_weights(checkpoint_filepath)
    
    %tensorboard --logdir logs/fit
    
    return history

In [ ]:
# Visualize metrics with graphs
def evaluate_model(history):
    fig, ax = plt.subplots(1, 4, figsize=(20, 3))
    ax = ax.ravel()

    for i, metric in enumerate(['accuracy', 'loss', 'precision', 'recall']):
        ax[i].plot(history.history[metric])
        ax[i].plot(history.history['val_' + metric])
        ax[i].set_title('Model {}'.format(metric))
        ax[i].set_xlabel('Epochs')
        ax[i].set_ylabel(metric.capitalize())
        ax[i].legend(['Training', 'Validation'])

In [ ]:
# Randomly pick and display an unseen image for the network to predict
def select_testing_image():
    image_num = random.randint(testing_set_start_index, testing_set_end_index)
    testing_img = testing_generator[0][0][image_num]

    class_num = np.argmax(testing_generator[0][1][image_num])
    true_label = classes[class_num].capitalize()

    print(f'True Label: {true_label}')

    return testing_img

In [ ]:
# Run a diagnosis to determine if the patient is COVID-19 positive or negative
def run_diagnosis(model, testing_img):      
    img_array = img_to_array(testing_img)
    img_array = img_array.reshape(-1, img_width, img_height, 3)

    predictions = model.predict(img_array)
    class_num = predictions.argmax()

    network_percent_confidence = str(np.max(predictions) * 100)[:4] + '% match'
    network_prediction = classes[class_num].capitalize()

    font = {'family': 'DejaVu Sans',
            'color' : 'red',
            'weight': 'heavy',
            'size'  :  10}
    
    plt.imshow(testing_img)
    # Find a way to only make the text after "Network Prediction" red
    plt.title(f'Network Prediction: {network_prediction} ({network_percent_confidence})', fontdict=font) 

In [ ]:
model = create_model()
compile_model(model)
print_model_summary(model)

In [ ]:
history = fit_model(model)

In [ ]:
evaluate_model(history)

In [ ]:
# Prints labels (alphabetical)
training_generator.class_indices

In [ ]:
def convert_to_img_array(img_path):
    # img is a PIL image
    img = load_img(img_path,
                   target_size=target_size)
    
    # Convert to float32 Numpy array
    img_array = img_to_array(img)
    img_array = preprocess_input(img_array)
    # img_array = img_array.reshape((-1, img_width, img_height, 3))

    # We add a dimension to transform our array into a "batch"
    # img_array = np.expand_dims(img_array, axis=0)

    return img_array

In [ ]:
# ! pip install git+git://github.com/raghakot/keras-vis.git --upgrade --no-deps
# import vis

# from vis.utils import utils
# from vis.visualization import visualize_cam

# --------------------------------------------
# FROM https://github.com/raghakot/keras-vis/blob/master/applications/self_driving/visualize_attention.ipynb
# --------------------------------------------
# for i, modifier in enumerate(modifiers):
#     heatmap = visualize_cam(model, layer_idx=-1, filter_indices=0, 
#                             seed_input=bgr_img, grad_modifier=modifier)
#     plt.figure()
#     plt.title(titles[i])
#     # Overlay is used to alpha blend heatmap onto img.
#     jet_heatmap = np.uint8(cm.jet(heatmap)[..., :3] * 255)
#     plt.imshow(overlay(img, jet_heatmap, alpha=0.7))

In [ ]:
# classifier_layer_names = ['top_conv', 'top_bn', 'top_activation', 'global_average_pooling2d_1', 'dropout_1', 'dense_3', 'dense_4', 'dense_5']
# last_conv_layer_name = 'block7d_add'

# img_path = f'/content/drive/My Drive/Colab Notebooks/COVID-19 Diagnosis/covid19_xray_dataset/Testing/covid19/Github COVID-19 X-ray Dataset/000001-1.jpg'

# img = load_img(img_path,
#                target_size=target_size)

# img               = img_to_array(img)
# # img               = preprocess_input(img)
# y_pred            = model.predict(img[np.newaxis,...])
# class_idxs_sorted = np.argsort(y_pred.flatten())[::-1]
# # topNclass         = 5
# # for i, idx in enumerate(class_idxs_sorted[:topNclass]):
# #     print("Top {} predicted class:     Pr(Class={:18} [index={}])={:5.3f}".format(
# #           i + 1,classlabel[idx],idx,y_pred[0,idx]))

# # Utility to search for layer index by name. 
# # ***********  Alternatively we can specify this as -1 since it corresponds to the last layer.  ***************
# layer_idx = utils.find_layer_idx(model, 'dense_8')
# # Swap softmax with linear
# model.layers[layer_idx].activation = tf.keras.activations.linear
# model = utils.apply_modifications(model)

# penultimate_layer_idx = utils.find_layer_idx(model, "top_conv") 
# class_idx  = class_idxs_sorted[0]
# seed_input = img
# grad_top1  = visualize_cam(model, layer_idx, class_idx, seed_input, 
#                            penultimate_layer_idx = penultimate_layer_idx) # TRY TO LEAVE BLANK? -> OR AS 'none'?
#                           #  backprop_modifier     = None)
#                           #  grad_modifier         = None)

In [ ]:
# def plot_map(grads):
#     fig, axes = plt.subplots(1,2,figsize=(14,5))
#     axes[0].imshow(_img)
#     axes[1].imshow(_img)
#     i = axes[1].imshow(grads,cmap="jet", alpha=0.8)
#     fig.colorbar(i)
#     plt.suptitle("Pr(class={}) = {:5.2f}".format(
#                       classlabel[class_idx],
#                       y_pred[0,class_idx]))

In [ ]:
testing_generator = testing_data_gen.flow_from_directory(ct_testing_dir,
                                                         target_size = target_size,
                                                         class_mode='binary',
                                                         batch_size=batch_size,
                                                         shuffle=False)

print(testing_generator.labels)

print(model.evaluate(testing_generator))

predictions = model.predict(testing_generator)
print(predictions)

In [ ]:
# # Test internet images (not reliable)
# ct_testing_dir = f'/content/drive/My Drive/Colab Notebooks/COVID-19 Diagnosis/Internet Images'

# testing_generator = testing_data_gen.flow_from_directory(ct_testing_dir,
#                                                          target_size = target_size,
#                                                          class_mode='binary',
#                                                          batch_size=batch_size,
#                                                          shuffle=False)

# print(testing_generator.labels)

# predictions = model.predict(testing_generator)

# i = 0;

# font = {'family': 'DejaVu Sans',
#             'color' : 'red',
#             'weight': 'heavy',
#             'size'  :  10}

# for pred in predictions:
#     print(str(i + 1) + ": " + filepath)

#     # Select the image array from the DirectoryIterator
#     img_array = testing_generator[0][0][i]
#     # Compress the array
#     # img_array = img_array.reshape(1, img_width, img_height, 3)
    
#     true_label_class_index = testing_generator.labels[index]
#     true_label = classes[true_label_class_index]

#     # network_percent_confidence = str(np.max(pred) * 100)[:4] + '% match'
    
#     # Safe prediction for test positive
#     # MAKE INTO CONSTANTS!!!!!!!!!!
#     if pred > 0 and pred < 0.3:
#         true_label_class_index = 0
#         network_percent_confidence = str(((0.5 - pred) / 0.5) * PERCENTAGE_FACTOR)[1:5]
#         print(network_percent_confidence)
#         print()

#     elif pred < 1 and pred > 0.7:
#         true_label_class_index = 0
#         network_percent_confidence = str(((pred - 0.5) / 0.5) * PERCENTAGE_FACTOR)[1:5]
#         print(network_percent_confidence)
#         print()

#     else:
#         # RETURN
#         print("Uncertain")
#         print()
    
#     network_prediction = classes[true_label_class_index]
    

#     plt.imshow(img_array)

#     print('True Label', true_label)
#     print(pred)
#     # print(f'Network Prediction: {network_prediction}')
    
#     # Find a way to only make the text after "Network Prediction" red
#     plt.title(f'{network_prediction} ({network_percent_confidence} % match)', fontdict=font)

#     i += 1

In [ ]:
# Test internet images (not reliable)
ct_testing_dir = f'/content/drive/My Drive/Colab Notebooks/COVID-19 Diagnosis/Internet Images'

testing_generator = testing_data_gen.flow_from_directory(ct_testing_dir,
                                                         target_size = target_size,
                                                         class_mode='binary',
                                                         batch_size=batch_size,
                                                         shuffle=False)

print(testing_generator.labels)

font = {'family': 'DejaVu Sans',
            'color' : 'red',
            'weight': 'heavy',
            'size'  :  10}

# Loop thorough all the images stored in the DirectoryIterator
for i in range(len(testing_generator[0][0])):
    print(str(i + 1) + ": " + testing_generator.filepaths[i])

    # Select an image array
    testing_image = testing_generator[0][0][i]
    img_array = testing_image
    
    # Compress the array
    img_array = img_array.reshape(1, img_width, img_height, 3)
    
    true_label_class_index = testing_generator.labels[i]
    # true_label_class_index = testing_generator[0][1][i]
    true_label = classes[true_label_class_index]

    prediction = model.predict(img_array)
    
    # Safe prediction for test positive
    # MAKE INTO CONSTANTS!!!!!!!!!!
    if prediction > 0 and prediction < COVID_THRESHOLD:
        network_percent_confidence = str(((EQUIDDST_MEAS - prediction) / EQUIDDST_MEAS) * PERCENTAGE_FACTOR)[1:5]
        print(network_percent_confidence)
        print()

    elif prediction < 1 and prediction > NORMAL_THRESHOLD:
        network_percent_confidence = str(((prediction - EQUIDDST_MEAS) / EQUIDDST_MEAS) * PERCENTAGE_FACTOR)[1:5]
        print(network_percent_confidence)
        print()

    else:
        # RETURN
        print('Uncertain')
        print()
    
    network_prediction = classes[true_label_class_index]
  
    print('True Label:', true_label)
    print('Prediction:', prediction)
    
    plt.title(f'{network_prediction} ({network_percent_confidence} % match)', fontdict=font)
    plt.imshow(img_array)
    plt.show()

In [ ]:
# print((testing_generator[0][0][-4]))
# print(len(testing_generator[0][1]))

# img_array = img_to_array(testing_generator[0][0][0])
img_array = testing_generator[0][0][0].reshape(1, img_width, img_height, 3)

# print(len(img_array))
# print(testing_generator[0][0])
index_num = 1

# for val in testing_generator.__getitem__(0):
#     print(str(index_num) + ': ' + str(val))
#     index_num += 1

# for val in testing_generator[0][0][0]:
#     print(str(index_num) + ': ' + str(val))
#     index_num += 1

print(type(testing_generator.filepaths))

GRAD-Cam

In [ ]:
# img_path = f'/content/drive/My Drive/Colab Notebooks/COVID-19 Diagnosis/covid19_xray_dataset/Testing/covid19/Github COVID-19 X-ray Dataset/000001-1.jpg'
# plot_map(grad_top1)

# Upload files to test
# from google.colab import files
# uploaded = files.upload()

# for filename in uploaded.keys():
#     # print('User uploaded file "{name}" with length {length} bytes'.format(
#     #     name=filename, length=len(uploaded[filename])))

#     img_path = filename
#     img = load_img(img_path,
#                    target_size=target_size)
#     img_array = img_to_array(img)
#     img_array = np.expand_dims(img_array, axis=0)
#     print(img_array)

    # testing_images = np.vstack([img_array])
    # predictions = model.predict(testing_generator)
    # class_pred = predictions.argmax(axis=-1)
    # print(predictions)
    # print(class_pred)

# testing_img = select_testing_image()
# run_diagnosis(model, testing_img)
# # From COVIDNet-CT: https://github.com/haydengunraj/COVIDNet-CT/blob/master/run_covidnet_ct.py
# print('**DISCLAIMER**')
#             print('Do not use this prediction for self-diagnosis. '
#                   'You should check with your local authorities for '
#                   'the latest advice on seeking medical assistance.')

In [ ]:
# #@title Enter Filepath Here:
# filename = "/content/chest_xray/val/PNEUMONIA/person1946_bacteria_4874.jpeg" #@param {type:"string"}

# img = image.load_img(filename, 
#                      target_size=(150, 150))
# x = image.img_to_array(img)
# x = np.expand_dims(x, axis=0)
# x = preprocess_input(x)

# y = final_model.predict(x)

# predicton="Normal" if y.argmax(axis=-1)==0 else "Pneumonia"
# actual="Normal" if "NORMAL" in filename else "Pneumonia" 

# img=mpimg.imread(filename)
# title_text = ("%s%s%s%s%s"%("True Label: ", actual, "\n", "Prediction: ", predicton))
# plt.title(title_text)
# imgplot=plt.imshow(img)